# Temperature forecast and history data example (Turku Artukainen)

Using Python 3.6

FMI open data https://en.ilmatieteenlaitos.fi/open-data

In [1]:
import datetime
import pandas as pd
import re
import requests
import xml.etree.ElementTree as et
from io import StringIO

api_url = 'https://opendata.fmi.fi/wfs'

## Forecast data

In [2]:
def get_fmidata_multipointcoverage(parameters):
    r = requests.get(f'{api_url}?{parameters}')
    # XML root and namespaces
    root = et.fromstring(r.text)
    namespaces = dict([node for _, node in et.iterparse(StringIO(r.text), events=['start-ns'])])
    # Extract name list
    names = list(map(lambda f: f.attrib['name'] ,root.findall('.//swe:field', namespaces)))
    # Extract Unix timestamps
    timestamps = re.split(r'\s+',root.find('.//gmlcov:positions', namespaces).text)[3:-1:3]
    # Convert Unix timestamps to datetimes with Helsinki timezone
    datetimeindex = pd.to_datetime(sorted(timestamps*len(names)), unit='s')
    datetimeindex = datetimeindex.tz_localize(tz='UTC').tz_convert('Europe/Helsinki')
    # Extract data
    values = re.split(r'\s+',root.find('.//gml:doubleOrNilReasonTupleList', namespaces).text)[1:-1]
    # Get URL for and print property explanations
    property_url = root.find('.//om:observedProperty', namespaces).attrib[
        '{http://www.w3.org/1999/xlink}href']
    print(f'Properties: {property_url}')
    # Create and return DataFrame
    df = pd.DataFrame({
        'name': names*len(timestamps), 
        'value': values}, 
        index=datetimeindex)
    return df

In [3]:
# Get geoids from https://www.geonames.org
geoid = 660972 # Turku, Artukainen
# List of stored queries https://ilmatieteenlaitos.fi/tallennetut-kyselyt
query = 'fmi::forecast::hirlam::surface::point::multipointcoverage'
df = get_fmidata_multipointcoverage(f'request=getFeature&storedquery_id={query}&geoid={geoid}')
temperature = df[df['name']=='Temperature']
temperature

Properties: https://opendata.fmi.fi/meta?observableProperty=forecast&param=GeopHeight,Temperature,Pressure,Humidity,WindDirection,WindSpeedMS,WindUMS,WindVMS,MaximumWind,WindGust,DewPoint,TotalCloudCover,WeatherSymbol3,LowCloudCover,MediumCloudCover,HighCloudCover,Precipitation1h,PrecipitationAmount,RadiationGlobalAccumulation,RadiationLWAccumulation,RadiationNetSurfaceLWAccumulation,RadiationNetSurfaceSWAccumulation,RadiationDiffuseAccumulation,LandSeaMask&language=eng


,name,value
2019-04-30 11:00:00+03:00,Temperature,10.9
2019-04-30 12:00:00+03:00,Temperature,12.03
2019-04-30 13:00:00+03:00,Temperature,12.52
2019-04-30 14:00:00+03:00,Temperature,11.95
2019-04-30 15:00:00+03:00,Temperature,11.23
2019-04-30 16:00:00+03:00,Temperature,10.57
2019-04-30 17:00:00+03:00,Temperature,10.21
2019-04-30 18:00:00+03:00,Temperature,9.66
2019-04-30 19:00:00+03:00,Temperature,9.09
2019-04-30 20:00:00+03:00,Temperature,8.56


## History data

In [4]:
def get_fmidata_simple(parameters):
    r = requests.get(f'{api_url}?{parameters}')
    # XML root and namespaces
    root = et.fromstring(r.text)
    namespaces = dict([node for _, node in et.iterparse(StringIO(r.text), events=['start-ns'])])
    # Extract datetimes, names and values
    datetimes = list(map(lambda f: f.text, root.findall('.//BsWfs:Time', namespaces)))
    names = list(map(lambda f: f.text, root.findall('.//BsWfs:ParameterName', namespaces)))
    values = list(map(lambda f: f.text, root.findall('.//BsWfs:ParameterValue', namespaces)))
    # Convert to Helsinki timezone
    datetimeindex = pd.to_datetime(datetimes).tz_convert('Europe/Helsinki')
    # Create and return DataFrame
    df = pd.DataFrame({
        'name': names, 
        'value': values}, 
        index=datetimeindex
    )
    return df

In [5]:
# Get geoids from https://www.geonames.org
geoid2 = 660972 # Turku, Artukainen
# Set time interval and timestep
end = datetime.datetime.utcnow().replace(second=0, microsecond=0)
start = end - datetime.timedelta(days=1)
timestep = 60 # minutes
query2 = 'fmi::observations::weather::simple'
df2 = get_fmidata_simple(f'request=getFeature&storedquery_id={query2}&geoid={geoid2}&starttime={start}&endtime={end}&timestep={timestep}')
history = df2[df2['name']=='t2m']
history

,name,value
2019-04-29 11:00:00+03:00,t2m,10.6
2019-04-29 12:00:00+03:00,t2m,11.4
2019-04-29 13:00:00+03:00,t2m,12.9
2019-04-29 14:00:00+03:00,t2m,12.8
2019-04-29 15:00:00+03:00,t2m,14.0
2019-04-29 16:00:00+03:00,t2m,14.3
2019-04-29 17:00:00+03:00,t2m,13.5
2019-04-29 18:00:00+03:00,t2m,13.4
2019-04-29 19:00:00+03:00,t2m,12.8
2019-04-29 20:00:00+03:00,t2m,12.0
